**Algoritmo de Agrupamento Kmeans**

**Código baseado em:**

1) https://www.kaggle.com/code/khotijahs1/k-means-clustering-of-iris-dataset

2) https://realpython.com/k-means-clustering-python/

In [57]:
# %pip install plotly --upgrade
# %pip install kneed 
# %pip install pandas
# %pip install seaborn
# %pip install scikit-learn
# %pip install nltk
# # To install only knee-detection algorithm

In [58]:
import pandas as pd
import nltk
import re
import string
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [59]:
nltk.download('punkt')
nltk.download('stopwords')
base= pd.read_csv('ReutersGrain-train.csv', sep=None)
base = base.drop_duplicates()
base

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\T-GAMER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\T-GAMER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_11136\2665828295.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  base= pd.read_csv('ReutersGrain-train.csv', sep=None)


,Text,class-att
0,'BAHIA COCOA REVIEW Showers continued througho...,0
1,'NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESE...,1
2,'ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS...,1
3,'CHAMPION PRODUCTS &lt.CH> APPROVES STOCK SPLI...,0
4,'COMPUTER TERMINAL SYSTEMS &lt.CPML> COMPLETES...,0
...,...,...
1549,'METEX CORP &lt.MTX> 4TH QTR DEC 28 Shr 22 cts...,0
1550,'HOUSTON OIL TRUST &lt.HO> OMITS MARCH DISTRIB...,0
1551,'MONOCLONAL ANTIBODIES &lt.MABS> BUYS COMPANY ...,0
1552,'U.S. TREASURY PROPOSES SOME S. AFRICAN IMPORT...,0


In [60]:
test = pd.read_csv('ReutersGrain-test.csv', sep = None)
test

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_11136\4041079289.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  test = pd.read_csv('ReutersGrain-test.csv', sep = None)


,Text,class-att
0,'ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN R...,0
1,'CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN ST...,1
2,'JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNW...,0
3,'THAI TRADE DEFICIT WIDENS IN FIRST QUARTER Th...,1
4,'INDONESIA SEES CPO PRICE RISING SHARPLY Indon...,0
...,...,...
599,'CORN SUPPLY/DEMAND BY COUNTRY -- USDA The U.S...,1
600,'USDA REPORTS EXPORT SALES ACTIVITY The U.S. A...,1
601,'INTERNATIONAL AMERICAN &ltHOME> TO ACQUIRE CO...,0
602,'SONY CHAIRMAN FORECASTS LOWER PROFITS THIS YE...,0


In [61]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [62]:
def preprocess(text):
    #Remover Numeros
    text= re.sub(r'\d+', '', text)
    #Remover pontuacao e outros marcadores
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    stemmer = SnowballStemmer('english')
   # print(text.lower())
    tokens = nltk.word_tokenize(text.lower(), preserve_line=False)
#Remoção de stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
#Stemming
    tokens = [stemmer.stem(token) for token in tokens]
#Reune os tokens em uma string
    processed_text = ' '.join(tokens)
    return processed_text

In [63]:
base['Text'][0]

"'BAHIA COCOA REVIEW Showers continued throughout the week in\\nthe Bahia cocoa zone, alleviating the drought since early\\nJanuary and improving prospects for the coming temporao,\\nalthough normal humidity levels have not been restored,\\nComissaria Smith said in its weekly review.\\n    The dry period means the temporao will be late this year.\\n    Arrivals for the week ended February 22 were 155,221 bags\\nof 60 kilos making a cumulative total for the season of 5.93\\nmln against 5.81 at the same stage last year. Again it seems\\nthat cocoa delivered earlier on consignment was included in the\\narrivals figures.\\n    Comissaria Smith said there is still some doubt as to how\\nmuch old crop cocoa is still available as harvesting has\\npractically come to an end. With total Bahia crop estimates\\naround 6.4 mln bags and sales standing at almost 6.2 mln there\\nare a few hundred thousand bags still in the hands of farmers,\\nmiddlemen, exporters and processors.\\n    There are doubt

In [64]:
preprocess(base['Text'][0])
for i in base.index:
    base['Text'][i] = preprocess(base['Text'][i])
base

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_11136\522829401.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  base['Text'][i] = preprocess(base['Text'][i])
C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_11136\522829401.py:3: FutureWarning

,Text,class-att
0,bahia cocoa review shower continu throughout w...,0
1,nation averag price farmerown reserv us agricu...,1
2,argentin grainoilse registr argentin grain boa...,1
3,champion product ltch approv stock split champ...,0
4,comput termin system ltcpml complet sale compu...,0
...,...,...
1549,metex corp ltmtx th qtr dec shr cts vs ctsn ne...,0
1550,houston oil trust ltho omit march distribut ho...,0
1551,monoclon antibodi ltmab buy compani monoclon a...,0
1552,us treasuri propos african import treasuri pro...,0


In [65]:
for i in test.index:
    test['Text'][i] = preprocess(test['Text'][i])
test    

C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_11136\4147159057.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test['Text'][i] = preprocess(test['Text'][i])
C:\Users\T-GAMER\AppData\Local\Temp\ipykernel_11136\4147159057.py:2: SettingWith

,Text,class-att
0,asian export fear damag usjapan rift mount tra...,0
1,china daili say vermin eat pct grain stock sur...,1
2,japan revis longterm energi demand downward mi...,0
3,thai trade deficit widen first quarter thailan...,1
4,indonesia see cpo price rise sharpli indonesia...,0
...,...,...
599,corn supplydemand countri usda us agricultur d...,1
600,usda report export sale activ us agricultur de...,1
601,intern american lthome acquir cos intern ameri...,0
602,soni chairman forecast lower profit year soni ...,0


In [66]:
vectorizer = CountVectorizer(analyzer = "word")
textos = vectorizer.fit_transform(base['Text'])
modelo = MultinomialNB()
modelo.fit(textos, base['class-att'])

MultinomialNB()

In [67]:
freq_testes = vectorizer.transform(test['Text'])
resultados = modelo.predict(freq_testes)

In [68]:
metrics.accuracy_score(test['class-att'], resultados)

0.9486754966887417

In [76]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(base['Text'])
sequences = tokenizer.texts_to_sequences(base['Text'])
data = pad_sequences(sequences,maxlen=100)
model = Sequential()
model.add(Embedding())
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(data, base['class-att'], epochs=10, batch_size=2)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sequential_10_1/embedding_7_1/GatherV2 defined at (most recent call last):
  File "c:\Users\T-GAMER\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\T-GAMER\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\T-GAMER\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\T-GAMER\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "C:\Users\T-GAMER\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\T-GAMER\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\T-GAMER\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 600, in run_forever

  File "c:\Users\T-GAMER\AppDat